# Data Cleaning

## Objective
Load the merged raw data (all 4 years) and clean it ONCE to create the final df_all dataset.

## Load Merged Raw Data from Notebook 01

In [122]:
import pandas as pd

# Load the merged raw data
all_stocks = pd.read_csv('../Data/Processed/nse_merged_raw.csv')

print(f"Loaded merged raw data: {all_stocks.shape}")
all_stocks.head()

Loaded merged raw data: (69945, 14)


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
0,04-Jan-21,EGAD,Eaagads Ltd,8.2,14,12.5,12.5,12.5,12.5,-,-,"3,200",-,Agricultural
1,04-Jan-21,KUKZ,Kakuzi Plc,300,397,365,365,365,365,-,-,-,-,Agricultural
2,04-Jan-21,KAPC,Kapchorua Tea Kenya Plc,59,90,78,78,78,78,-,-,-,-,Agricultural
3,04-Jan-21,LIMT,Limuru Tea Plc,360,475,360,360,360,360,-,-,100,-,Agricultural
4,04-Jan-21,SASN,Sasini Plc,14.8,20,19.5,19.5,19.5,19.5,-,-,-,-,Agricultural


In [123]:
all_stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69945 entries, 0 to 69944
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            69945 non-null  object
 1   Code            69945 non-null  object
 2   Name            69945 non-null  object
 3   12m Low         69945 non-null  object
 4   12m High        69945 non-null  object
 5   Day Low         69945 non-null  object
 6   Day High        69945 non-null  object
 7   Day Price       69945 non-null  object
 8   Previous        69945 non-null  object
 9   Change          69945 non-null  object
 10  Change%         69945 non-null  object
 11  Volume          69945 non-null  object
 12  Adjusted Price  69945 non-null  object
 13  Sector          69754 non-null  object
dtypes: object(14)
memory usage: 7.5+ MB


In [124]:
#sort percentage of missing values and display data types
(all_stocks.isna().mean() * 100).sort_values(ascending=False).to_frame(
    name='Percentage_missing').assign(Dtype=all_stocks.dtypes).style.bar(
        subset=['Percentage_missing'], color="purple")

,Percentage_missing,Dtype
Sector,0.273072,object
Date,0.000000,object
Code,0.000000,object
Name,0.000000,object
12m Low,0.000000,object
12m High,0.000000,object
Day Low,0.000000,object
Day High,0.000000,object
Day Price,0.000000,object
Previous,0.000000,object


In [125]:
#Check for duplicates
all_stocks.duplicated().sum()

0

Cleaning steps:

    1. clean numerical columns since all columns are categorized as object.

     - Remove % in change percent column and also commas in numbers like in Volume column 

    2. Convert Date column to datetime and extract Month and Year for analysis

    3. Rename columns to standardize them for consistency

    4. handle missing values

    5. Remove Indices like ^NASI, ^N25I on Code column.

    5. Create a copy of final cleaned dataset and save the cleaned dataset

## Clean Numeric Columns

Convert all price, volume, and percentage columns to numeric.

In [126]:
# Columns to clean
cols = [
    "12m Low", "12m High", "Day Low", "Day High",
    "Day Price", "Previous", "Change", "Change%",
    "Volume", "Adjusted Price"
]

# Remove % sign from Change%
all_stocks["Change%"] = all_stocks["Change%"].astype(str).str.replace("%", "", regex=False)

# Remove commas, replace '-' with 0, convert to numeric
all_stocks[cols] = (
    all_stocks[cols]
        .replace(",", "", regex=True)
        .replace("-", 0)
        .apply(pd.to_numeric, errors="coerce")
)

In [127]:
all_stocks.head()

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
0,04-Jan-21,EGAD,Eaagads Ltd,8.2,14.0,12.5,12.5,12.5,12.5,0.0,0.0,3200.0,0.0,Agricultural
1,04-Jan-21,KUKZ,Kakuzi Plc,300.0,397.0,365.0,365.0,365.0,365.0,0.0,0.0,0.0,0.0,Agricultural
2,04-Jan-21,KAPC,Kapchorua Tea Kenya Plc,59.0,90.0,78.0,78.0,78.0,78.0,0.0,0.0,0.0,0.0,Agricultural
3,04-Jan-21,LIMT,Limuru Tea Plc,360.0,475.0,360.0,360.0,360.0,360.0,0.0,0.0,100.0,0.0,Agricultural
4,04-Jan-21,SASN,Sasini Plc,14.8,20.0,19.5,19.5,19.5,19.5,0.0,0.0,0.0,0.0,Agricultural


In [128]:
all_stocks.tail()

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
69940,31-Dec-24,^N25I,NSE 25-Share Index,2364.29,3457.87,3402.80,3402.80,3402.80,3380.24,22.56,0.67,0.0,0.0,Indices
69941,31-Dec-24,^NASI,NSE All-Share Index,90.10,125.34,123.48,123.48,123.48,123.12,0.36,0.29,0.0,0.0,Indices
69942,31-Dec-24,^NBDI,NSE Bonds Index,986.46,1108.71,1107.41,1107.41,1107.41,1093.87,13.54,1.24,0.0,0.0,NaN
69943,31-Dec-24,^ZKEQTK,Zamara Kenya Equity Index (KES),1535.23,2547.58,2519.80,2519.80,2519.80,2495.28,24.52,0.98,0.0,0.0,Indices
69944,31-Dec-24,^ZKEQTU,Zamara Kenya Equity Index (USD),818.46,1697.07,1677.27,1677.27,1677.27,1662.24,15.03,0.90,0.0,0.0,Indices


## Converting Date Column

In [129]:
## Converting Date Column
all_stocks['Date'] = pd.to_datetime(all_stocks['Date'], errors='coerce')

print(f"Date range: {all_stocks['Date'].min()} to {all_stocks['Date'].max()}")

Date range: 2021-01-04 00:00:00 to 2024-12-31 00:00:00


C:\Users\User\AppData\Local\Temp\ipykernel_31776\10312273.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_stocks['Date'] = pd.to_datetime(all_stocks['Date'], errors='coerce')


In [130]:
#extract month and year
all_stocks["Month"]= all_stocks["Date"].dt.month

all_stocks["Year"]= all_stocks["Date"].dt.year

## Renaming Columns for Consistency and Remove '^' from stock_code column


In [131]:
def clean_and_standardize(df):
    df = df.copy()
    
    # 1️⃣ Standardize column names
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(' ', '_', regex=False)
        .str.title()
    )
    
    # 2️⃣ Rename specific columns
    df = df.rename(columns={
        'Code': 'Stock_Code',
        'Change%': 'Percentage_Change'
    })
    
    # 3️⃣ Remove ^ from Stock_Code column (if it exists)
    if 'Stock_Code' in df.columns:
        df['Stock_Code'] = df['Stock_Code'].str.replace('^', '', regex=False)
    
    return df


all_stocks = clean_and_standardize(all_stocks)


In [132]:
#check if "^" is removed
all_stocks['Stock_Code'].unique()[:10]


array(['EGAD', 'KUKZ', 'KAPC', 'LIMT', 'SASN', 'WTK', 'CGEN', 'ABSA',
       'BKG', 'COOP'], dtype=object)

## Remove missing values

In [133]:
#drop missing values in sector column
all_stocks.dropna(subset=["Sector"],inplace=True)

## Create df_all - Final Cleaned Dataset

In [134]:
# Create df_all
df_all = all_stocks.copy()

# Sort by Stock_code and Date
df_all = df_all.sort_values(['Stock_Code', 'Date']).reset_index(drop=True)

df_all.head()

,Date,Stock_Code,Name,12M_Low,12M_High,Day_Low,Day_High,Day_Price,Previous,Change,Percentage_Change,Volume,Adjusted_Price,Sector,Month,Year
0,2021-01-04,ABSA,ABSA Bank Kenya Plc,8.5,14.2,9.42,9.80,9.52,9.66,-0.14,1.45,18500.0,0.0,Banking,1,2021
1,2021-01-05,ABSA,ABSA Bank Kenya Plc,8.5,14.2,9.44,9.70,9.44,9.52,-0.08,0.84,1923300.0,0.0,Banking,1,2021
2,2021-01-06,ABSA,ABSA Bank Kenya Plc,8.5,14.2,9.40,9.68,9.44,9.44,0.00,0.00,233400.0,0.0,Banking,1,2021
3,2021-01-07,ABSA,ABSA Bank Kenya Plc,8.5,14.2,9.36,9.46,9.40,9.44,-0.04,0.42,194700.0,0.0,Banking,1,2021
4,2021-01-11,ABSA,ABSA Bank Kenya Plc,8.5,14.2,9.44,9.70,9.46,9.48,-0.02,0.21,77900.0,0.0,Banking,1,2021


## Data Quality Checks

In [135]:
#final check to see if cleaned
#sort percentage of missing values and display data types
(all_stocks.isna().mean() * 100).sort_values(ascending=False).to_frame(
    name='Percentage_missing').assign(Dtype=all_stocks.dtypes).style.bar(
        subset=['Percentage_missing'], color="purple")

c:\Users\User\anaconda3\anac3\envs\learn-env\lib\site-packages\pandas\io\formats\style.py:3851: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Percentage_missing,Dtype
Date,0.000000,datetime64[ns]
Stock_Code,0.000000,object
Name,0.000000,object
12M_Low,0.000000,float64
12M_High,0.000000,float64
Day_Low,0.000000,float64
Day_High,0.000000,float64
Day_Price,0.000000,float64
Previous,0.000000,float64
Change,0.000000,float64


In [136]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69754 entries, 0 to 69753
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               69754 non-null  datetime64[ns]
 1   Stock_Code         69754 non-null  object        
 2   Name               69754 non-null  object        
 3   12M_Low            69754 non-null  float64       
 4   12M_High           69754 non-null  float64       
 5   Day_Low            69754 non-null  float64       
 6   Day_High           69754 non-null  float64       
 7   Day_Price          69754 non-null  float64       
 8   Previous           69754 non-null  float64       
 9   Change             69754 non-null  float64       
 10  Percentage_Change  69754 non-null  float64       
 11  Volume             69754 non-null  float64       
 12  Adjusted_Price     69754 non-null  float64       
 13  Sector             69754 non-null  object        
 14  Month 

In [137]:
all_stocks.columns

Index(['Date', 'Stock_Code', 'Name', '12M_Low', '12M_High', 'Day_Low',
       'Day_High', 'Day_Price', 'Previous', 'Change', 'Percentage_Change',
       'Volume', 'Adjusted_Price', 'Sector', 'Month', 'Year'],
      dtype='object')

## Save Cleaned Data

In [138]:
# Save cleaned data
df_all.to_csv('../Data/processed/cleaned_nse.csv', index=False)



print(f"  nse_all_clean.csv: {df_all.shape}")


  nse_all_clean.csv: (69754, 16)


## 📊 Final Cleaned Dataset Summary

The dataset has been thoroughly cleaned and finalized.
It contains 69,754 rows and 16 columns.

### Column Descriptions:

**Date** – The calendar date of the trading activity.

**Stock_Code** – A unique stock ticker symbol identifying each listed security.

**Name** – The official name of the company or index.

**12M_Low** – The lowest price at which the stock traded over the past 12 months.

**12M_High** – The highest price recorded over the past 12 months.

**Day_Low** – The lowest trading price recorded on that specific day.

**Day_High** – The highest trading price recorded on that specific day.

**Day_Price** – The closing price of the stock on that trading day.

**Previous** – The closing price from the previous trading day.

**Change** – The absolute difference between the current day’s closing price and the previous day’s closing price.

**Percentage_Change** – The percentage change between the current and previous closing prices.

**Volume** – The total number of shares traded on that day.

**Adjusted_Price** – The stock price adjusted for corporate actions such as dividends or stock splits.

**Sector** – The economic sector to which the company belongs.

**Month** – The month extracted from the trading date (for time-series analysis).

**Year** – The year extracted from the trading date (for trend analysis).